# Marketing Assistant

Copyright 2025, Denis Rothman

**Goal:** This notebook serves as the practical runtime for the **Strategic Marketing Engine**, as architected in Chapter 9. It demonstrates how the generic, multi-domain "Glass Box" Context Engine can be seamlessly repurposed to solve a variety of real-world marketing challenges without any changes to its core code.

This notebook will:
* Connect to the Pinecone knowledge base populated by the `Data_Ingestion_Marketing.ipynb` script.
* Use the generic Control Deck templates to interact with the engine.
* Execute the marketing use cases

*Note* Make sure to run `Data_Ingestion_Marketing.ipynb` first to ingest the data for this notebook.


# I. Inititalization

## GitHub

In [1]:
print("Downloading files from public repository...")

# The -f flag tells curl to fail on an error (like 404)
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/helpers.py --output helpers.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/agents.py --output agents.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/registry.py --output registry.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/engine.py --output engine.py
# (You might want to add a check here to see if the files actually exist now)
print("✅ File download attempt finished!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1823  100  1823    0     0   6510      0 --:--:-- --:--:-- --:--:--  6534
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6479  100  6479    0     0  23483      0 --:--:-- --:--:-- --:--:-- 23474
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9440  100  9440    0     0  29181      0 --:--:-- --:--:-- --:--:-- 29226
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3611  100  3611    0     0  12704      0 --:--:-- --:--:-- --:--:-- 12714
  % Total    % Received % Xferd  Average Speed   Tim

## Installation and client setup

In [2]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [3]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [4]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
import json  # <--- Added for type handling
from IPython.display import display, Markdown

# In Legal_Compliance_Assistant.ipynb (The "Engine Room" cell)

def execute_and_display(goal, config, client, pc, moderation_active):
    """
    Runs the context engine, now with robust type handling for moderation.
    """
    # --- PRE-FLIGHT MODERATION CHECK (on user input) ---
    if moderation_active:
        print("--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---")
        # Goal is always a string, so this is safe
        moderation_report = helpers.helper_moderate_content(text_to_moderate=goal, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Goal failed pre-flight moderation. Execution halted.")
            return # Halt execution before calling the engine

    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config
    )

    # --- POST-FLIGHT MODERATION CHECK (on AI output) ---
    if result and moderation_active:
        print("\n--- [Safety Guardrail] Performing Post-Flight Moderation Check on Output ---")

        # HANDLE NON-STRING OUTPUTS ---
        # If the agent returns a dictionary (JSON), convert it to a string first.
        text_to_check = result
        if isinstance(result, (dict, list)):
            try:
                text_to_check = json.dumps(result, indent=2)
            except TypeError:
                text_to_check = str(result)
        # ------------------------------------------

        moderation_report = helpers.helper_moderate_content(text_to_moderate=text_to_check, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Generated output failed post-flight moderation and will be redacted.")
            result = "[Content flagged as potentially harmful by moderation policy and has been redacted.]"

    # 2. Display the Final Result
    print("\n--- FINAL OUTPUT ---")
    if result:
        # If result is still a dict (and wasn't redacted), display it nicely
        if isinstance(result, dict):
             display(Markdown(f"```json\n{json.dumps(result, indent=2)}\n```"))
        else:
             display(Markdown(str(result)))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace
    print("\n\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # --- THIS LINE IS TO CREATE THE 'pp' OBJECT ---
        pp = pprint.PrettyPrinter(indent=2)

        pp.pprint(trace.steps)

## Control Deck configuration

In [5]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#III.CONTROL DECKS

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [6]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.
goal = "Summarize the key points of the QuantumDrive"

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---
Moderation Report:
{'categories': {'harassment': False,
                'harassment/threatening': False,
                'harassment_threatening': False,
                'hate': False,
                'hate/threatening': False,
                'hate_threatening': False,
                'illicit': False,
                'illicit/violent': False,
                'illicit_violent': False,
                'self-harm': False,
                'self-harm/instructions': False,
                'self-harm/intent': False,
                'self_harm': False,
                'self_harm_instructions': False,
                'self_harm_intent': False,
                'sexual': False,
                'sexual/minors': False,
                'sexual_minors': False,
                'violence': False,
                'violence/graphic': False,
                'violence_graphic': False},
 'flagged': False,
 'scores': {'harassment': 4

```json
{
  "summary": "- QuantumDrive is a high-speed NVMe solid-state drive (SSD) built for professional creatives (video editors, 3D artists, photographers) to remove storage bottlenecks.\n\n- It uses flash memory (no moving parts) with the NVMe 2.0 protocol over PCIe Gen 5, delivering up to 7,500 MB/s read and 7,000 MB/s write speeds, so large files and projects load and save very quickly.\n\n- An integrated graphene heat spreader keeps the drive cool and prevents thermal throttling, so performance stays fast and consistent during long, heavy workloads like rendering and batch exports.\n\n- It includes AES\u2011256 hardware encryption and the DataWeaver software suite for secure storage and easy backup/encryption management.\n\n- Available in 2 TB, 4 TB, and 8 TB capacities with a 5\u2011year warranty and high endurance (e.g., 3,000 TBW for the 4 TB model), it\u2019s positioned as the single most impactful upgrade to speed up professional creative workflows."
}
```



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 25.15 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'QuantumDrive: summary and synthesis '
                                       'based on the provided documents\n'
                                       '\n'
                                       '**Core concept**  \n'
                                       'QuantumDrive Q-1 is a high-performance '
                                       'NVMe solid-state drive (SSD) designed '
                                       'specifically to remove storage '
                                       'bottlenecks in professional creative '
                                       'workflows (video editing, 3D '
                                       'rendering, photography). It’s '
                                       'positioned as the “single most '
                                       'impactful upgrade” for a cre

In [7]:
#@title Product Marketing Copy Generation(Use Case 1)
goal = "Analyze the ChronoTech press release and summarize their core product messaging and value proposition. Please cite your sources."


# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Chrono SSD Pro is ChronoTech’s new flagship SSD built for people who hate waiting on their gear. It runs on PCIe Gen 4 and hits up to 7,300 MB/s read speeds, so loading, saving, and moving massive files (video, 3D, big photo projects) feels instant.

The whole idea is “Speed for the Modern Creator” — basically, remove storage bottlenecks so your workflow doesn’t get interrupted and you can stay in the zone. No more staring at progress bars while your inspiration fades.

It’s performance‑first, super fast, and still good value in the premium space, especially the 4TB model, which gives you top-tier speed without a ridiculous price tag. If you’re tired of waiting on your storage to catch up with your ideas, Chrono SSD Pro is meant to be the answer.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 28.48 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'ChronoTech’s most recent core product '
                                       'messaging, positioning, and value '
                                       'proposition (from the referenced press '
                                       'release) centers on the **Chrono SSD '
                                       'Pro** and can be summarized as '
                                       'follows:\n'
                                       '\n'
                                       '---\n'
                                       '\n'
                                       '### Core Product Messaging\n'
                                       '\n'
                                       '- **Product**: Chrono SSD Pro – '
                                       'ChronoTech’s *new flagship solid-state '
           

In [8]:
#@title Product Marketing Copy Generation(Use Case 2)


# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

## 1. Define the Goal: A research query that asks for a creative output.
goal = "Using the official product spec sheet, write a short marketing description for the new QuantumDrive Q-1. The description should be confident, aspirational, and focus on the benefits for creative professionals. Please cite your sources."



# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


This is a high‑end NVMe PCIe Gen 5 SSD built for serious creative work.

- Comes in 2 TB, 4 TB, and 8 TB, so it can handle big video projects, 3D assets, and massive photo libraries.
- It’s fast: up to 7,500 MB/s reads and 7,000 MB/s writes. That means quicker project loading, smooth scrubbing through 4K+ footage, and faster renders/exports.
- The 4 TB model is rated for 3,000 TBW, so it can handle tons of writes (renders, previews, cache) without dying early, plus there’s a 5‑year limited warranty.
- Has a graphene heat spreader to keep temps under control, so performance doesn’t nosedive during long renders or big file transfers.
- Security-wise, you get AES‑256 hardware encryption and a DataWeaver backup/encryption suite to protect client work and simplify backups.
- Overall, it’s aimed at pro editors, 3D artists, and photographers who want to cut down render and load times while keeping their workstation stable and secure.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 21.75 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'Below is a consolidated, spec‑style '
                                       'overview of the QuantumDrive Q‑1 based '
                                       'solely on the provided materials.\n'
                                       '\n'
                                       '---\n'
                                       '\n'
                                       '## QuantumDrive Q‑1 – Official Product '
                                       'Specification Overview\n'
                                       '\n'
                                       '**Product Name:** QuantumDrive Q‑1  \n'
                                       '**Product Type:** Solid-State Drive '
                                       '(SSD)  \n'
                                       '**Target Market:** Creative '
               

In [9]:
#@title Writing a brand pitch recommendation(Use case 3)

# 1. Define the Goal: A creative or factual task that is deliberately
#    outside the scope of the documents in the knowledge base.
#    - DOMAIN: Universal test applicable to any curated knowledge base.
#    - KEY CAPABILITY: Tests the `Researcher` agent's ability to report a
#      negative finding and the `Writer` agent's ability to handle it gracefully,
#      preventing hallucination.
# goal = "[INSERT YOUR OUT-OF-SCOPE GOAL HERE]"


# =CONTROL DECK 3: A persuasive pitch ===
goal = "Write a persuasive pitch on our brand tone and voice guide"


# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Having a clear brand tone and voice guide is basically like giving your whole company one shared “personality manual.” Here’s what it does in plain terms:

- **Everything sounds like the same brand**  
  Whether it’s social posts, emails, or web pages, the language and vibe stay consistent. That makes the brand easier to recognize and trust.

- **Content gets made way faster**  
  Writers aren’t guessing about how to sound. Clear rules (simple language, no fluff, no hedging) let teams crank out aligned campaigns without back-and-forth.

- **You stand out more**  
  A distinct voice—confident, expert, but still human—helps you feel different from competitors without trash-talking them.

- **People trust you more**  
  Direct claims, real proof (like testimonials and warranties), and no wild promises make the brand feel credible instead of hype-y.

- **Marketing, product, and sales tell the same story**  
  Everyone talks about the product in the same way: focused on customer goals and outcomes, not just features and specs.

- **Better customer experience overall**  
  Clear, jargon-free, easy-to-skim content makes every touchpoint smoother—from an Instagram ad to a long email. The tone acknowledges pain points and then shows a better future.

- **More effective marketing**  
  Because the focus is on benefits (speed, reliability, better workflow) instead of tech jargon, the messaging hits what customers actually care about—and usually performs better.

Bottom line: a solid tone and voice guide makes your brand feel consistent, trustworthy, and memorable, while helping your teams move faster and tell one clear, compelling story from first touch to purchase.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 25.60 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Summarize information '
                                  'quickly and casually.", "style_guide": "Use '
                                  'informal language. Keep it brief and '
                                  'engaging. Imagine explaining it to a '
                                  'friend.", "instruction": "Summarize the '
                                  'provided facts using the casual style '
                                  'guide."}'},
    'planned_input': { 'intent_query': 'Persuasive, professional, '
                                       'marketing-style pitch that explains '
                                       'the value of a brand tone and voice '
                                       'guide and motivates internal '
                                       'stakeholders to use